Autor: Matyáš Sládek <br>
Rok: 2020 <br>

Tento soubor slouží k trénování a klasifikaci datových sad s možností použití všech sad atributů a parametrů. Dále je možné natrénované klasifikátory uložit a načíst a také zobrazit či uložit matice predikcí.

Tato buňka importuje potřebné knihovny.

In [8]:
%%capture
import os
import sys
import time
from datetime import timedelta
import json
import joblib

import IPython.display as ipd
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter
import seaborn as sns
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import confusion_matrix, f1_score

# Classifiers
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

from tqdm.notebook import tqdm
tqdm().pandas()

Tato buňka obsahuje funkci pro načtení a zpracování dat, trénování, načítání a ukládání klasifikačních algoritmů a klasifikaci.

In [9]:
def perform_classification(dataset, library, classifiers, default_params, parameters):
    '''
    Function loads and processes required data and performs training and classification.
    Trained classifiers can be saved and loaded and confusion matrices shown or saved.
    
    Parameters:
    
    optimised_feature_sets: Dictionary containing optimised feature sets
    dataset:                Name of a dataset to indicate which extracted features to load (dataset from which the features were extracted)
    library:                Name of a library to indicate which extracted features to load (library with which the features were extracted)
    classifiers:            Dictionary containing classifiers for which the optimal feature subset should be found
    default_params:         Dictionary containing parameters for classifiers
    parameters:             Parameters about which selection to use etc.
    '''
    
    def classify():
        '''
        Inner function that performs training and classification.
        Trained classifiers can be saved and loaded and confusion matrices shown or saved.
        This function has access to parameters in scope of the outer function.
        '''
        
        clfs = {}   # Stores classifier objects

        if parameters['use_default_hyper_parameters']:   # If selected, create unique name for classifier with parameters default and initialize classifier with default parameters
                                                         # or set object to None if pretrained classifier should be loaded
            try:
                clfs[classifier_name + '_default'] = None if parameters['load_pretrained_classifiers'] else classifier(**default_params[classifier_name])
            except KeyError:
                print('Default parameters for classifier {} not found!'.format(classifier_name), file=sys.stderr)

        if parameters['use_optimised_hyper_parameters_CV']:   # If selected, create unique name for classifier with parameters optimised using cross-validation and initialize classifier with default parameters
                                                              # or set object to None if pretrained classifier should be loaded
            try:
                clfs[classifier_name + '_optimised_CV'] = None if parameters['load_pretrained_classifiers'] else classifier(**default_params[classifier_name], **optimised_parameters[dataset][library][feature_set_name][classifier_name]['CV'])
            except KeyError:
                print('{} {} {} {} {} Optimised hyper parameters not found!'.format(dataset, library, feature_set_name, classifier_name, 'CV'), file=sys.stderr)

        if parameters['use_optimised_hyper_parameters_VS']:   # If selected, create unique name for classifier with parameters optimised using validation set and initialize classifier with default parameters  
                                                              # or set object to None if pretrained classifier should be loaded
            try:
                clfs[classifier_name + '_optimised_VS'] = None if parameters['load_pretrained_classifiers'] else classifier(**default_params[classifier_name], **optimised_parameters[dataset][library][feature_set_name][classifier_name]['VS'])
            except KeyError:
                print('{} {} {} {} {} Optimised hyper parameters not found!'.format(dataset, library, feature_set_name, classifier_name, 'VS'), file=sys.stderr)
                
        for clf_name, clf in clfs.items():   # For each of the selected classifiers
            t = time.time()   # Store start time of the optimisation process
            
            if parameters['load_pretrained_classifiers']:   # Load pretrained classifiers if selected
                try:
                    clf = joblib.load('../metadata/trained_classifiers/{}_{}_{}_{}.joblib.dat'.format(dataset, library, feature_set_name, clf_name))
                except Exception as e:
                    print('Failed to read file: "../metadata/trained_classifiers/{}_{}_{}_{}.joblib.dat"'.format(dataset, library, feature_set_name, clf_name), file=sys.stderr)
                    print('Error: {}'.format(repr(e)), file=sys.stderr)
                    continue
            else:
                clf.fit(X_train, y_train)   # Train the classifier
                
                if parameters['save_trained_classifiers']:   # Save the trained classifier if selected
                    
                    if not parameters['use_test_set']:
                        print('Saving classifiers should be done when test set is selected to use all training data.', file=sys.stderr)
                    
                    joblib.dump(clf, '../metadata/trained_classifiers/{}_{}_{}_{}.joblib.dat'.format(dataset, library, feature_set_name, clf_name)) 
            
            # Perform classification either on test set or validation set using F1 or accuracy scoring
            if parameters['use_test_set']:               
                if parameters['use_f1_score']:
                    score = f1_score(y_test, clf.predict(X_test), average='macro')
                else:
                    score = clf.score(X_test, y_test)
            else:
                if parameters['use_f1_score']:
                    score = f1_score(y_val, clf.predict(X_val), average='macro')
                else:
                    score = clf.score(X_val, y_val)
                
            scores.loc[feature_set_name, clf_name] = score   # Save classification score to dataframe
            runtimes.loc[feature_set_name, clf_name] = str(timedelta(seconds=(time.time() - t))).split(".")[0]   # Save training and classification time to dataframe

            # Show or save confusion matrix if selected
            if parameters['save_confusion_matrices'] or parameters['show_confusion_matrices']:
                cm = confusion_matrix(y_test, clf.predict(X_test), normalize='true')   # Create normalized confusion matrix
                df_cm = pd.DataFrame(cm, index = encoder.classes_, columns = encoder.classes_)   # Create dataframe from confusion matrix with correct classes on axes
                fig, ax = plt.subplots(figsize = (10,7))   # Create matplotlib figure
                ax.set_title('{} {} {} {} confusion matrix'.format(dataset, library, feature_set_name, clf_name))   # Set title to figure
                fmt = lambda x,pos: '{:.0%}'.format(x)   # Format colorbar values to percentage
                ax = sns.heatmap(df_cm, cbar_kws={'format': FuncFormatter(fmt)}, annot=True, fmt=".2%")   # Create heatmap from confusion matrix with annotated values (percentage format)
                ax.set(xlabel="Predicted label", ylabel = "True label")   # Name axes
                plt.tight_layout()   # Correct layout to fit figure

                if parameters['save_confusion_matrices']:   # Save confusion matrix to PDF if selected
                    
                    # Create folder for storing confusion matrices
                    if not os.path.exists('../metadata/confusion_matrices'):
                        try:
                            os.mkdir('../metadata/confusion_matrices')
                        except Exception as e:
                            print_log('{}: {}'.format('os.mkdir(../metadata/confusion_matrices)', repr(e)), file=sys.stderr)
                    
                    plt.savefig("../metadata/confusion_matrices/{}_{}_{}_{}_{}.pdf".format(dataset, library, feature_set_name, clf_name, 'TS' if parameters['use_test_set'] else 'VS'))

                if not parameters['show_confusion_matrices']:   # Close plot to not be shown automatically if selected
                    plt.close()
                    
        return
        
        
    # If only test set is selected to be used            
    if parameters['load_test_set']:
        
        # Automatically select correct parameter values if not selected already
        parameters['use_test_set'] = True
        parameters['load_pretrained_classifiers'] = True
        parameters['save_trained_classifiers'] = False
        
        feature_set_name = 'all'   # Set correct feature set name for the test sets (only test sets with all features can be used due to scaling)
        scores = pd.DataFrame(index=[feature_set_name])   # Init dataframe to store scores
        runtimes = pd.DataFrame(index=[feature_set_name])   # Init dataframe to store prediction times
        X_test = pd.read_csv("../metadata/test_data/{}_X_test.csv".format(dataset), header=None)   # Load test data (already scaled)
        X_test = X_test.values   # Transform dataframe to ndarray

        genres = pd.read_csv("../metadata/test_data/{}_genres.csv".format(dataset), index_col=0, header=0)   # Load test data target genres
        encoder = LabelEncoder()   # Init label encoder
        y_test = encoder.fit_transform(np.ravel(genres))   # Encode genres
        
        # For each selected classifier perform classification
        for classifier_name, _ in classifiers.items():
            classify()
            
        return(scores, runtimes)
    
    optimised_feature_sets_names = []   # Stores the names of optimised feature sets to be used for classification
    X_val = None   # Stores the features of validation set if used
    y_val = None   # Stores the genres of validation set if used
    
    # If any of the optimised feature sets is selected
    if parameters['use_opt_feature_set_FS_CV'] or parameters['use_opt_feature_set_FS_VS'] or parameters['use_opt_feature_set_BE_CV'] or parameters['use_opt_feature_set_BE_VS']:
        
        # Load optimised feature sets if available
        try:
            with open('../metadata/misc/optimised_feature_sets.json') as f:
                optimised_feature_sets = json.load(f)   
        except Exception as e:
            print('Failed to read file: "../metadata/misc/optimised_feature_sets.json"!', file=sys.stderr)
            print('Error: {}'.format(repr(e)), file=sys.stderr)
            return -1
        
        # If feature set optimised with forward selection and cross-validation is selected, add it to the dictionary of feature sets to be optimised
        if parameters['use_opt_feature_set_FS_CV']:
            optimised_feature_sets_names.append('opt_feature_set_FS_CV')
            
        # If feature set optimised with forward selection and validation set is selected, add it to the dictionary of feature sets to be optimised
        if parameters['use_opt_feature_set_FS_VS']:
            optimised_feature_sets_names.append('opt_feature_set_FS_VS')
            
        # If feature set optimised with backward elimination and cross-validation is selected, add it to the dictionary of feature sets to be optimised
        if parameters['use_opt_feature_set_BE_CV']:
            optimised_feature_sets_names.append('opt_feature_set_BE_CV')
            
        # If feature set optimised with backward elimination and validation set is selected, add it to the dictionary of feature sets to be optimised
        if parameters['use_opt_feature_set_BE_VS']:
            optimised_feature_sets_names.append('opt_feature_set_BE_VS')

    # Load optimised hyper parameters
    if parameters['use_optimised_hyper_parameters_CV'] or parameters['use_optimised_hyper_parameters_VS']:
        try:
            with open('../metadata/misc/optimised_hyper_parameters.json') as f:
                optimised_parameters = json.load(f) 
        except Exception as e:
            print('Failed to read file: "../metadata/misc/optimised_hyper_parameters.json"!', file=sys.stderr)
            print('Error: {}'.format(repr(e)), file=sys.stderr)
            return -1
    
    # Load specified extracted features
    try:                
        features = pd.read_csv('../metadata/features/features_{}_{}.csv'.format(dataset, library), index_col=0, header=[0, 1, 2])        
    except Exception as e:
        print('Failed to read file: "../metadata/features/features_{}_{}.csv"!'.format(dataset, library), file=sys.stderr)
        print('Error: {}'.format(repr(e)), file=sys.stderr)
        return -1
        
    # Perform One-hot encoding on categorical features
    # This method is modified so that there is always full number of columns generated, even if not all cases of categorical features are present
    # This is redundant when EBD, FMA or GTZAN datasets are used, but when a smaller dataset would be used, 
    # it would ensure properly working predicitions with trained classifiers on data from a different dataset
    
    # Perform One-hot encoding on categorical features
    for column in features.select_dtypes(include='object'):   # For each categorical column
        dummy_columns = pd.get_dummies(features[column])   # Encode the column values   
        features = features.drop(columns=column)   # Drop the column from the dataframe

        # Reindex columns to fixed length with all possible instances to avoid feature mismatch with trained classifiers
        if (column[0] in ['chords_key', 'key_edma', 'key_krumhansl', 'key_temperley']) and (column[1] in ['none', 'key']):
            dummy_columns = dummy_columns.reindex(['Ab', 'B', 'Bb', 'C', 'C#', 'D', 'E', 'Eb', 'F', 'F#', 'G'], axis=1, fill_value=0)
        elif (column[0] in ['chords_scale', 'key_edma', 'key_krumhansl', 'key_temperley']) and (column[1] in ['none', 'scale']):
            dummy_columns = dummy_columns.reindex(['minor'], axis=1, fill_value=0)

        # Create correct multiindex for the encoded columns and append them to the features dataframe
        dummy_columns.columns = pd.MultiIndex.from_product([[column[0]], [column[1]], ['{}'.format(c) for c in dummy_columns.columns]], names=features.columns.names)
        features = pd.concat([features, dummy_columns], axis=1).sort_index(axis=1)
                
    feature_names = list(features.columns.levels[0])   # Get names of all features    
    feature_sets = {}
    
    # If all features is selected, add all feature names to the dictionary of feature sets to be used
    if parameters['use_all_features']:
        feature_sets['all'] = feature_names
        
    # Can be uncommented to use each feature separately to evaluate them etc.
#     for name in feature_names:
#         feature_sets[name] = name
            
    scores = pd.DataFrame(index=list(feature_sets.keys()) + optimised_feature_sets_names)   # Init dataframe to store classification scores
    runtimes = pd.DataFrame(index=list(feature_sets.keys()) + optimised_feature_sets_names)   # Init dataframe to store training and/or prediction runtimes
        
    # Load track-genre list
    try:                
        genres = pd.read_csv("../metadata/track_genre_lists/{}_track_genre_list.csv".format(dataset), index_col=0, header=0)        
    except Exception as e:
        print('Failed to read file: "../metadata/track_genre_lists/{}_track_genre_list.csv"!'.format(dataset), file=sys.stderr)
        print('Error: {}'.format(repr(e)), file=sys.stderr)
        return -1
    
    genres = genres.loc[features.index]   # Remove unwanted data from track-genre list (data about tracks removed from features because of corruption or other reasons)

    # Encode genre labels
    encoder = LabelEncoder()
    y = encoder.fit_transform(np.ravel(genres))

    scaler = StandardScaler()
        
    # For each of the non-optimised feature sets
    for feature_set_name, feature_set in feature_sets.items():
        X = features[feature_set].values   # Extract selected feature set values to ndarray
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)   # Split genres and features to train and test sets (stratified)
        
        if parameters['use_test_set']:   # If test set is selected, scale features according to values in whole train set
            X_train = scaler.fit_transform(X_train)
            X_test = scaler.transform(X_test)
        else:   # Else split train data to new train set and validation set and scale features according to values in the new train set
            X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=parameters['validation_set_size'], random_state=42, stratify=y_train)
            X_train = scaler.fit_transform(X_train)
            X_val = scaler.transform(X_val)

        # Perform classification for each selected classifier
        for classifier_name, classifier in classifiers.items():                            
            classify()
            
    # This loop is included because with optimised feature sets data have to be splitted for each classifier separately,
    # therefore having two separate loops for optimised and non-optimised feature sets is more efficient
    for feature_set_name in optimised_feature_sets_names:   # For each selected optimised feature set        
        for classifier_name, classifier in classifiers.items():   # For each selected classifier
            
            # Load optimised feature sets
            try:
                feature_set = optimised_feature_sets[dataset][library][classifier_name][feature_set_name]
            except KeyError:
                print('Optimised feature set {} for classifier {} not found!'.format(feature_set_name, classifier_name), file=sys.stderr)
                continue
            
            X = features[feature_set].values   # Extract selected feature set values to ndarray
            X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)    # Split genres and features to train and test sets (stratified)

            if parameters['use_test_set']:   # If test set is selected, scale features according to values in whole train set
                X_train = scaler.fit_transform(X_train)
                X_test = scaler.transform(X_test)
            else:      # Else split train data to new train set and validation set and scale features according to values in the new train set
                X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=parameters['validation_set_size'], random_state=42, stratify=y_train)
                X_train = scaler.fit_transform(X_train)
                X_val = scaler.transform(X_val)
            
            classify()   # Perform classification

    return(scores, runtimes)

Tato buňka slouží k nastavení potřebných parametrů a výběru datových sad, sad atributů, klasifikátorů a jejich parametrů, s pomocí kterých má být klasifikace provedena. <br>
Pro jaké datové sady má být klasifikace provedena je možné zvolit v proměnné <code>datasets</code> odkomentováním/zakomentováním příslušných záznamů. <br>
Při použití vlastní datové sady je nutné do proměnné <code>datasets</code> přidat stejný název datové sady, jako byl použit pro extrakci atributů v souboru <strong>feature_extraction.ipynb</strong> <br>
Dále je nutné zvolit, jestli klasifikaci provést na atributech extrahovaných pomocí knihovny Librosa či Essentia odkomentováním/zakomentováním příslušných záznamů v proměnné <code>feature_extraction_libraries</code>. <br>
Které klasifikátory mají být pro klasifikaci využity je možné zvolit v proměnné <code>classifiers</code> odkomentováním/zakomentováním příslušných záznamů. <br>
Při použití jiného klasifikátorů je tento nutné přidat do proměnné <code>classifiers</code> ve formátu {zvolený_název_klasifikátoru}:{odkaz_na_objekt_klasifikátoru}. <br>
Paremetry klasifikátorů je možné upravit v proměnné <code>default_params</code>, případně pro nový klasifikátor přidat záznam ve formátu {zvolený_název_klasifikátoru}:{slovník_parametrů}, kde zvolený název klasifikátoru musí odpovídat názvu v proměnné <code>classifiers</code>. <br>
V proměnné <code>parameters</code> je možné nastavit parametry klasifikace. <br>
<br>
Popis parametrů: <br>
<ul>
    <li><code>use_f1_score</code> Hodnota true značí použití F1 skóre, hodnota False klasické skóre (počet správných / počet celkem)</li>
    <li><code>load_test_set</code> Hodnota True značí použití uložené testovací sady, tato možnost automaticky zaručí použití celé sady atributů a načtení natrénovaných klasifikátorů nehledě na hodnoty parametrů</li>
    <li><code>use_test_set</code> Hodnota True snačí použití testovací sady, hodnota False použití validační sady</li>
    <li><code>validation_set_size</code> Značí, jaká část trénovacích dat bude vyhrazena jako validační, pokud je tato metoda zvolena</li>
    <li><code>use_all_features</code> Hodnota True znamená použití celé sady atributů</li>
    <li><code>use_opt_feature_set_FS_CV</code> Hodnota True znamená použití sady atributů vybrané pomocí metody dopředné selekce a křížové validace</li>
    <li><code>use_opt_feature_set_FS_VS</code> Hodnota True znamená použití sady atributů vybrané pomocí metody dopředné selekce a validace na validační saďe</li>
    <li><code>use_opt_feature_set_BE_CV</code> Hodnota True znamená použití sady atributů vybrané pomocí metody zpětné eliminace a křížové validace</li>
    <li><code>use_opt_feature_set_BE_VS</code> Hodnota True znamená použití sady atributů vybrané pomocí metody zpětné eliminace a validace na validační saďe</li>
    <li><code>use_default_hyper_parameters</code> Hodnota True značí použití neoptimalizovaných parametrů</li>
    <li><code>use_optimised_hyper_parameters_CV</code> Hodnota True značí použití optimalizovaných parametrů pomocí křížové validace</li>
    <li><code>use_optimised_hyper_parameters_VS</code> Hodnota True značí použití optimalizovaných parametrů pomocí validační sady</li>
    <li><code>save_trained_classifiers</code> Hodnota True značí uložení vybraných natrénovaných klasifikačních algoritmů na vybraných sadách atributů</li>
    <li><code>load_pretrained_classifiers</code> Hodnota True značí použití uložených natrénovaných klasifikátorů</li>
    <li><code>save_confusion_matrices</code> Hodnota true značí uložení matice predikcí do PDF pro každý vybraný klasifikátor na každé ze sad atributů</li>
    <li><code>show_confusion_matrices</code> Hodnota tru značí zobrazení matice predikcí pro každý vybraný klasifikátor na každé ze sad atributů</li>
</ul>

In [10]:
if __name__ == "__main__":
       
    def highlight_max(s):
        '''
        Hilight the maximum value in selected axis of a dataframe.
        '''
        is_max = s == s.max()
        return ['background-color: yellow' if v else '' for v in is_max]
        
    # List of datasets whose features should be used, unwanted can be commented out   
    datasets = [
#         'EBD',
        'FMA',
#         'GTZAN'
    ]
    
    # List of feature extraction libraries whose features should be used, unwanted can be commented out   
    feature_extraction_libraries = [
#         'librosa',
        'essentia'
    ]
    
    # Dictionary of classifier names and objects to use for classification, unwanted can be commented out   
    classifiers = {
        # sklearn classifiers
#         'LogisticRegression':LogisticRegression,
#         'KNeighborsClassifier':KNeighborsClassifier,
#         'MLPClassifier':MLPClassifier,
#         'DecisionTreeClassifier':DecisionTreeClassifier,
#         'SVC_linear':SVC,
#         'SVC_rbf':SVC,
        
        # sklearn ensemble classifiers
#         'RandomForestClassifier':RandomForestClassifier,
        
        # other classifiers
        'XGBClassifier':XGBClassifier,
    }
    
    # Default classifiers parameters to use 
    default_params = {
        'LogisticRegression': {'max_iter':10000, 'class_weight':'balanced'},
        'KNeighborsClassifier': {'n_jobs':-1, 'algorithm':'brute'},
        'MLPClassifier': {'max_iter':10000, 'random_state':42},
        'DecisionTreeClassifier': {'class_weight':'balanced', 'random_state':42},
        'SVC_linear': {'kernel':'linear', 'class_weight':'balanced'},
        'SVC_rbf': {'kernel':'rbf', 'class_weight':'balanced'},
        'RandomForestClassifier': {'n_jobs':-1, 'class_weight':'balanced', 'random_state':42},
        'XGBClassifier': {'tree_method':'gpu_hist', 'n_jobs':1, 'random_state':42},
    }
    
    parameters = {
        'use_f1_score': True,   # Set to true to use F1 men scoring, else use accuracy scoring
        'load_test_set': False,   # Loads test data, automatically select test set with all features to be classified with pretrained classifiers
        'use_test_set': True,   # Set to true to use test set, else a portion of the train data will be reserved as validation set
        'validation_set_size': 0.2,   # Determines the size of the portion of training data, which will be reserved as validation set, ignored if 'use_test_set' parameter is set to True
        'use_all_features': True,   # Use all available features extracted with selected library
        'use_opt_feature_set_FS_CV': False,   # Set this to true to use feature set optimised with forward selection and cross-validation
        'use_opt_feature_set_FS_VS': False,   # Set this to true to use feature set optimised with backward elimination and validation set
        'use_opt_feature_set_BE_CV': False,   # Set this to true to use feature set optimised with forward selection and cross-validation
        'use_opt_feature_set_BE_VS': False,   # Set this to true to use feature set optimised with backward elimination and validation set
        'use_default_hyper_parameters': False,   # Set this to True to use default hyper parameters
        'use_optimised_hyper_parameters_CV': False,   # Set this to True to use hyper parameters optimised using cross-validation
        'use_optimised_hyper_parameters_VS': True,   # Set this to True to use hyper parameters optimised using validation set
        'save_trained_classifiers': False,   # Save trained classifiers
        'load_pretrained_classifiers': True,   # Load pretrained classifiers
        'save_confusion_matrices': False,   # Save confusion matrix for each classifier
        'show_confusion_matrices': False,   # Display confusion matrix for each classifier
    }

    if parameters['use_test_set']:
        print('Using test set with {} scoring.'.format('F1' if parameters['use_f1_score'] else 'accuracy'))
    else:
        print('Using validation set. with {} scoring.'.format('F1' if parameters['use_f1_score'] else 'accuracy'))
    
    t_start = time.time()   # Store the start time of classification
    
    for dataset in datasets:   # For each of the selected datasets
        
        for library in feature_extraction_libraries:   # For each of the selected extraction libraries features
            
            t = time.time()   # Store the start time of classification for selected dataset and extraction library features
            scores, runtimes = perform_classification(dataset, library, classifiers, default_params, parameters)
            print("{} dataset classification using features extracted with {} library finished in {}.".format(dataset, library, str(timedelta(seconds=(time.time() - t))).split(".")[0]))

            # Create folder for storing confusion matrices
            if not (os.path.exists('../metadata/scores') and os.path.exists('../metadata/runtimes')):
                try:
                    os.mkdir('../metadata/scores')
                except Exception as e:
                    print_log('{}: {}'.format('os.mkdir(../metadata/scores)', repr(e)), file=sys.stderr)
                    
                try:
                    os.mkdir('../metadata/runtimes')
                except Exception as e:
                    print_log('{}: {}'.format('os.mkdir(../metadata/runtimes)', repr(e)), file=sys.stderr)
            
            scores.to_csv("../metadata/scores/scores_{}_{}_{}_{}.csv".format(dataset, library, 'test_set' if parameters['use_test_set'] else 'validation_set', 'F1_score' if parameters['use_f1_score'] else 'accuracy_score'))   # Save the classification scores to a file
            runtimes.to_csv("../metadata/runtimes/runtimes{}_{}_{}_{}.csv".format('_predictions' if parameters['load_pretrained_classifiers'] else '', dataset, library, 'test_set' if parameters['use_test_set'] else 'validation_set'))   # Save the classification runtimes to a file

            print("{} {} scores:".format(dataset, library))
            scores = scores.style.apply(highlight_max, axis=1)   # Apply higlight max styling to scores dataframe
            ipd.display(scores.format("{:.2%}"))   # Display the scores dataframe
            print("{} {} runtimes:".format(dataset, library))
            ipd.display(runtimes)   # Display the runtimes dataframe
    
    print("All datasets classification finished in {}.".format(str(timedelta(seconds=(time.time() - t_start))).split(".")[0]))

Using test set with F1 scoring.
FMA dataset classification using features extracted with essentia library finished in 0:00:09.
FMA essentia scores:


,XGBClassifier_optimised_VS
all,64.21%


FMA essentia runtimes:


,XGBClassifier_optimised_VS
all,0:00:00


All datasets classification finished in 0:00:09.
